In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import sleep
import os

### Get main page

In [2]:
main_page = 'https://www.winiary.pl'
response = requests.get(main_page)
soup = BeautifulSoup(response.text, 'html.parser')

### Find all recipes

In [3]:
recipe_divs = soup.find_all('div', {'class': 'box recipe fit-3'})
recipe_divs += soup.find_all('div', {'class': 'box recipe fit-3 more'})

In [4]:
recipes = dict()
for recipe in recipe_divs:
    recipe_a = recipe.find_all('a', href=True)[-1]
    recipe_url = recipe_a['href']
    recipe_name = recipe_a.text.strip()
    recipes[recipe_name] = recipe_url

### Scrap recipes

In [7]:
wrong = dict()
for name, url in recipes.items():
    if os.path.exists('data' + name + '.xlsx'):
        continue
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        span = soup.find_all('span', {'itemprop': 'recipeIngredient'})
        ingredients = [item.find('a').text.strip() for item in span]
        units = [item.find('span', {'class': 'unit'}).text.strip() for item in span]
        units = [unit[1:-1].replace('\xa0\r\n', ' ') for unit in units]
        instruction_list = soup.find_all('li', {'itemprop': 'recipeInstructions'})
        steps = [step.text.strip().replace('\xa0', ' ') for step in instruction_list]
        data = {'ingredients': ingredients, 'units': units, 'steps': steps}
        df = pd.DataFrame(dict([(key, pd.Series(values)) for key, values in data.items()]))
        df.to_excel('data/' + name + '.xlsx', index=False)
    except:
        print(name, url)
        wrong[name] = url
    sleep(3)

Tort urodzinowy WINIARY https://www.winiary.pl/przepis.aspx/139947/tort-urodzinowy-winiary
Placuszki z dyni https://www.winiary.pl/przepis.aspx/138407/placuszki-z-dyni
Żurek z zapiekanką ziemniaczaną, boczkiem, kapustą i majerankiem https://www.winiary.pl/przepis.aspx/140634/zurek-z-zapiekanka-ziemniaczana-boczkiem-kapusta-i-majerankiem
Ogórki w zalewie musztardowej https://www.winiary.pl/przepis.aspx/138304/ogorki-w-zalewie-musztardowej
Sernik z Princessą Zebrą https://www.winiary.pl/przepis.aspx/139996/sernik-z-princessa-zebra
Pasta z dyni do chleba https://www.winiary.pl/przepis.aspx/138421/pasta-z-dyni-do-chleba
Curry z dynią z Bulionem Warzywnym Winiary https://www.winiary.pl/przepis.aspx/139975/curry-z-dynia-z-bulionem-warzywnym-winiary
Rozpływający się w ustach stek wołowy z orzechami i gorgonzolą https://www.winiary.pl/przepis.aspx/140206/rozplywajacy-sie-w-ustach-stek-wolowy-z-orzechami-i-gorgonzola
Gnocchi z indykiem w pomidorach https://www.winiary.pl/przepis.aspx/139723/gno